In [2]:
import path_configs # noqa
import tensorflow as tf
import settings
from datetime import datetime
import pandas as pd
import os
from pathlib import Path
import matplotlib.pyplot as plt
import scipy
import numpy as np
import librosa
settings.init()
import seaborn as sns
sns.set(font_scale=1.5, font="Arial", style="white")
# from modules.ClassifierGenerators import (TrainClassifierGenerator,  # noqa
#                                           ValidationClassifierGenerator)  # noqa

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

TRAIN_DATA_LEN: 3714569
VAL_DATA_LEN: 1590319


In [3]:
#assumes that array is not zero
def scaled(tensor):
    return (tensor-tf.math.reduce_min(tensor))/(tf.math.reduce_max(tensor)-tf.
                                                math.reduce_min(tensor))
def scaled_array(array):
    return (array - np.min(array))/(np.max(array) - np.min(array))

import scipy.spatial.distance as dist
def similarity(x1, x2):
    x1 = librosa.feature.mfcc(x1).flatten()
    x2 = scipy.signal.resample(librosa.feature.mfcc(x2).flatten(), len(x1))
    distance = dist.correlation(x1, x2)
    return 1/(1+distance)

from modules.DataPreprocessor import DataLoader
dl = DataLoader()
sample_rate = 48000
window_time = dl.window_time
frame_length = int(48000*window_time/1000)
frame_step = frame_length//4
def reverse_to_audio(db_spec):
    audio_rev_spec = librosa.db_to_power(db_spec, ref=1.0)
    print(
        'reversing'
    )
    return librosa.feature.inverse.mel_to_audio(audio_rev_spec, sr=48000, n_fft=frame_length, hop_length=frame_length//4)

In [4]:
train = pd.read_csv('data_info.csv')

In [5]:
# from tqdm import tqdm
# for set_name in ['test', 'train', 'val']:
#     for gender_name in ['male', 'female']:
#         for age_name in ['teens', 'twenties','seventies', 'fifties', 'fourties','thirties', 'sixties', 'eighties']:        
#             Path(os.path.join('data', 'images',set_name, gender_name, age_name)).mkdir(parents=True, exist_ok=True)

# for spec_num in tqdm(np.arange(dl.train.shape[0]+1)):
#     spec = dl.make_spectrogram(spec_num)
#     paded_spec = dl.pad_spec(spec)
#     set_type = dl.train.iloc[spec_num, 7]
#     age = dl.train.iloc[spec_num, 2]
#     gender = dl.train.iloc[spec_num, 3]
#     for window in np.arange(0, paded_spec.shape[1]-64, 64):
#         spec_window = paded_spec[:, window:window + 128]
#         name = str('spec_' + str(window//64) + '_' + str(spec_num))
#         np.save(os.path.join('data', 'images',set_type, gender, age, name), spec_window, allow_pickle = False)

In [6]:
dl.train

,client_id,path,age,gender,sentence,length,length_cut,set_type
0,7eff9a54bdb0619deffda7609d5b8565278e3328de99e6...,common_voice_en_19956141.mp3,sixties,female,He had success right off the bat with Do You B...,266112,199680,train
1,3dbc57adea9742c6782b7c88b7ac313e4bb7e26374db2e...,common_voice_en_20012566.mp3,fourties,female,"While the Doctor aids Clent, Penley leaves the...",252288,233280,train
2,5b1dd93d15c860f4d94c18ca33bb896b1b4a4f2f95f253...,common_voice_en_18343993.mp3,teens,male,The hyena proceeded to dine.,161280,84480,train
3,70c21ca1cc3de05f3c23b11615cd7d5dd078e03cda0cd4...,common_voice_en_20791392.mp3,fifties,female,"In Manchuria and Siberia, they mate during Jan...",271872,216960,train
4,db94fba2e15010f1a4aa114d910ac1e1a320c5dbd692a3...,common_voice_en_20005534.mp3,fourties,male,The reconfigured centre would have been brande...,205056,149760,train
...,...,...,...,...,...,...,...,...
38725,4ba8b91d601ee7f27be3011552ea94c8d076d9816d4a77...,common_voice_en_19769331.mp3,seventies,female,The first volume was written during Ovid's jou...,288000,165120,train
38726,afcc38200b25a04019f5e3125535f1d93674ad6f0be9ac...,common_voice_en_193416.mp3,sixties,female,Play Lil Hardin Armstrong from Itunes.,253440,120960,train
38727,e140a69135f35bc81e98140a267e18a382807d684099be...,common_voice_en_21169446.mp3,twenties,male,It is primarily a farming community.,196992,86400,val
38728,64bb74229c8a05aeeebe8aeb498d5a0389479625a0085d...,common_voice_en_19761030.mp3,twenties,male,The Ewings had thirteen children.,171648,168960,val


In [7]:
def append_to_TFRecord(writer: tf.io.TFRecordWriter, x_dict: dict, y_dict: dict) -> None:
    """
    Append data to open writer.
    :param writer: TFRecordWriter
    :param x_dict: dict with np.arrays
    :param y_dict: dict with np.arrays
    :return: None
    """
    def _bytes_feature(value):
        return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
    features = dict()
    for key in x_dict.keys():
        features[key] = _bytes_feature(tf.compat.as_bytes(x_dict[key].astype(np.float32).tostring()))
    for key in y_dict.keys():
        features[key] = _bytes_feature(tf.compat.as_bytes(y_dict[key].astype(np.float32).tostring()))
    example = tf.train.Example(features=tf.train.Features(feature=features))
    writer.write(example.SerializeToString())

In [ ]:
from tqdm import tqdm
path = os.path.join('data', 'tf_record')
file_name = 'data_file_train.tfrecord'
Path(path).mkdir(parents=True, exist_ok=True)
options = tf.io.TFRecordOptions(compression_level=1, compression_type="ZLIB")

with tf.io.TFRecordWriter(os.path.join(path, file_name), options=options) as writer:
    for spec_num in tqdm(np.arange(dl.train.shape[0]+1)):
        spec = dl.make_spectrogram(spec_num)
        paded_spec = dl.pad_spec(spec)
        set_type = dl.train.iloc[spec_num, 7]
        age = dl.train.iloc[spec_num, 2]
        gender = dl.train.iloc[spec_num, 3]
        for window in np.arange(0, paded_spec.shape[1]-64, 64):
            spec_window = paded_spec[:, window:window + 128]
            x_dict = {'x': np.array(spec_window)}
            y_dict = {'y_age':np.array(age), 'y_gender':np.array(gender)}
            append_to_TFRecord(writer=writer, x_dict=x_dict, y_dict=y_dict)


In [6]:
# pathlist = Path(os.path.join('data', 'images')).rglob('*.npy')
# path_list = []
# for path in pathlist:
#     path_list.append(str(path))

# df = pd.DataFrame({'path':path_list})
# df['set_type'] = df['path'].apply(lambda string: os.path.normpath(string).split(os.sep)[2])
# df['gender'] = df['path'].apply(lambda string: os.path.normpath(string).split(os.sep)[3])
# df['age'] = df['path'].apply(lambda string: os.path.normpath(string).split(os.sep)[4])

# df = df.sample(frac = 1).reset_index(drop=True)
# df.to_csv(os.path.join('data', 'images', 'image_metadata.csv'), index=False)

In [9]:
lengths_cut = []
for spec_num in tqdm(np.arange(38781+1)):
    audio = np.array(dl.load_audio_binary(spec_num))
    cut_audio = dl.cut_voice(audio)
    lengths_cut.append(len(cut_audio))

100%|██████████| 38782/38782 [44:11<00:00, 14.63it/s]


In [14]:
np.sum(lengths_cut)/dl.train.length.sum()

0.6941762549697098

In [17]:
train[train.age == 'teens'].gender.value_counts()

male      3713
female    2285
Name: gender, dtype: int64

In [18]:
train.gender.value_counts( )

female    20005
male      18777
Name: gender, dtype: int64

In [10]:
from scipy.io.wavfile import write

scaled = np.int16(audio/np.max(np.abs(audio)) * 32767)
write('testOG.wav', 48000, scaled)

scaled2 = np.int16(cut_audio/np.max(np.abs(cut_audio)) * 32767)
write('test_recovered.wav', 48000, scaled2)